In [1]:
from torchvision.datasets import MNIST, mnist, CIFAR10
from torchvision import transforms

In [2]:
import torch.nn.functional as F

In [3]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.distributions import Categorical
from torch.utils.data import DataLoader

In [4]:
from tqdm.notebook import tqdm_notebook

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from itertools import chain
import pandas as pd

In [7]:
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
class CustomTargetTransform:
    def __init__(self, num_classes=10):
        self.num_classes = num_classes

    def __call__(self, target):
        new_target = torch.zeros(self.num_classes, dtype=torch.float, device=device)
        new_target[target] = 1
        return new_target

transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Lambda(lambda x: x.float().to(device))
])

# data_loader = DataLoader(dataset, batch_size=800, shuffle=True)

In [9]:
dataset = mnist.FashionMNIST("data", download=True, train=True, transform=transform, target_transform=CustomTargetTransform())
dataset_target = mnist.FashionMNIST("data", download=True, train=False, transform=transforms.PILToTensor())
# dataset = CIFAR10("data", download=True, train=True, transform=transform, target_transform=CustomTargetTransform())
# dataset_target = CIFAR10("data", download=True, train=False, transform=transforms.PILToTensor())

In [10]:
# target_data = torch.tensor(dataset_target.data).swapaxes(3, 1).float().to(device)
target_data = torch.tensor(dataset_target.data).unsqueeze(1).float().to(device)
target_labels = torch.tensor(dataset_target.targets).float().to(device)
target_data.shape, target_labels.shape

/tmp/ipykernel_4927/3376170973.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_data = torch.tensor(dataset_target.data).unsqueeze(1).float().to(device)
/tmp/ipykernel_4927/3376170973.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_labels = torch.tensor(dataset_target.targets).float().to(device)


(torch.Size([10000, 1, 28, 28]), torch.Size([10000]))

In [11]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, batch_norm1, batch_norm2, inter_channels, kernel_size1, kernel_size2, stride1, stride2, padding1=1, padding2=1, in_size=28):
        super().__init__()
        block = [
            nn.Conv2d(in_channels, inter_channels, kernel_size1, stride1, padding1),
        ]
        if batch_norm1:
            block.append(nn.BatchNorm2d(inter_channels))
        block.append(nn.ReLU())
        block.append(nn.Conv2d(inter_channels, in_channels, kernel_size2, stride2, padding2))
        if batch_norm2:
            block.append(nn.BatchNorm2d(in_channels))
        self.block = nn.Sequential(
            *block
        )
    
    def get_out_size(self, in_size):
        for layer in (self.block[0], self.block[-1] if type(self.block[-1]) == nn.Conv2d else self.block[-2]):
            in_size = (in_size - layer.kernel_size[0] + 2 * layer.padding[0]) // layer.stride[0] + 1
        return in_size

    def forward(self, x):
        return F.relu(x + self.block(x))


In [12]:
def get_name(model: nn.Module) -> list[nn.Module]:
    skip = 1
    names = []
    for m in model.modules():
        if skip > 0:
            skip -= 1
            continue
        names.append(m)
        if len(tuple(m.modules())) > 1:
            skip = len(tuple(m.modules())) - 1
    return names

In [13]:
def create_model(
        img_size,
        pool_kernel_size,
        pool_stride,
        *layers,
        linears: list[int],
        lr=0.01,
        batch_size=800,
        epoch_count=10,
        softmax=True,
    ) -> tuple[nn.Sequential, nn.CrossEntropyLoss, torch.optim.SGD, SummaryWriter]:
    torch.manual_seed(0)
    blocks = []
    in_channels = 1
    outs = img_size
    for layer in layers:
        blocks.append(
            layer
        )
        tpe = type(layer)
        if tpe == ResBlock:
            outs = layer.get_out_size(outs)
        elif tpe == nn.Conv2d:
            outs = (outs - layer.kernel_size[0] + 2 * layer.padding[0]) // layer.stride[0] + 1
            in_channels = layer.out_channels
        elif tpe == nn.MaxPool2d:
            outs = (outs - layer.kernel_size) // layer.stride + 1
    outs = (outs - pool_kernel_size) // pool_stride + 1
    outs = outs * outs * in_channels
    blocks.append(nn.MaxPool2d(kernel_size=pool_kernel_size, stride=pool_stride))
    blocks.append(nn.Flatten(1))
    for layer in linears:
        blocks.append(nn.Linear(outs, layer))
        outs = layer
    blocks.append(nn.Linear(outs, 10))
    if softmax:
        blocks.append(nn.Softmax())
    model = nn.Sequential(
        *blocks,
    ).to(device)
    er_f = nn.CrossEntropyLoss()
    optim = torch.optim.AdamW(model.parameters(), lr=lr)
    # name = "_".join(m._get_name() for m in model.modules())
    # name = f"_{len(blocks_out_channels)}_{lr}_{blocks_kernel_size}_{blocks_stride}_{pool_kernel_size}_{pool_stride}"
    name = "_".join(m._get_name() for m in get_name(model))
    print(name)
    writer = SummaryWriter(comment=name)
    return model, er_f, optim, writer, name, batch_size, epoch_count


In [14]:
def write_summary(writer: SummaryWriter, model: nn.Module):
    param_dict = {f"layer_{i}": str(m) for m, i in enumerate(get_name(model))}
    param_table = pd.DataFrame(param_dict, index=[0])
    predicted = model(target_data)
    predicted_labels = predicted.max(1).indices
    confusion_matrix = torch.zeros(10, 10, device=device)
    for i in range(len(predicted_labels)):
        confusion_matrix[predicted_labels[i].long()][target_labels[i].long()] += 1
    confusion_matrix = confusion_matrix.cpu()
    fig = plt.gcf()
    fig.clear()
    ax = fig.add_subplot(111)
    hist = predicted_labels[predicted_labels == target_labels]
    error_hist = target_labels[predicted_labels != target_labels]
    percent: torch.tensor = ((predicted_labels == target_labels).sum() / len(target_labels))
    sns.heatmap(confusion_matrix, annot=True, fmt="g", ax=ax)
    writer.add_figure("confusion_matrix", fig)
    writer.add_scalar("accuracy", percent)
    fig.clear()
    ax = fig.add_subplot(111)
    sns.histplot(hist.cpu(), stat="count", discrete=True, bins=range(10), ax=ax)
    writer.add_figure("right hist", fig)
    fig.clear()
    ax = fig.add_subplot(111)
    sns.histplot(error_hist.cpu(), stat="count", discrete=True, bins=range(10), ax=ax)
    writer.add_figure("error hist", fig)
    writer.add_text("param_table", param_table.to_markdown())
    writer.add_text("accuracy", str(percent.item()))

In [57]:
models = (
    # create_model(
    #     28,
    #     2,
    #     2,
    #     nn.Conv2d(1, 3, 3, 1, 1, 1),
    #     ResBlock(3, 3, 3, 3, 1, 1),
    #     nn.Conv2d(3, 6, 3, 1, 1, 1),
    #     ResBlock(6, 6, 3, 3, 1, 1),
    #     lr=0.1,
    #     batch_size=800,
    #     epoch_count=15,
    # ), # 74 - 251
    # create_model(
    #     28,
    #     2,
    #     2,
    #     nn.Conv2d(1, 3, 3, 1, 1, 1),
    #     ResBlock(3, False, False, 3, 3, 3, 1, 1),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(3, 6, 3, 1, 1, 1),
    #     ResBlock(6, False, False, 6, 3, 3, 1, 1),
    #     lr=0.1,
    #     batch_size=800,
    #     epoch_count=10,
    # ), # 79 - 500
    # create_model(
    #     28,
    #     2,
    #     2,
    #     nn.Conv2d(1, 3, 3, 1, 1, 1),
    #     ResBlock(3, 3, 3, 3, 1, 1),
    #     nn.ReLU(),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(3, 6, 3, 1, 1, 1),
    #     ResBlock(6, 6, 3, 3, 1, 1),
    #     lr=0.1,
    #     batch_size=800,
    #     epoch_count=10,
    # ), # 70 - 300
    # create_model(
    #     28,
    #     2,
    #     2,
    #     nn.Conv2d(1, 3, 3, 1, 1, 1),
    #     ResBlock(3, 3, 3, 3, 1, 1),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(3, 6, 3, 1, 1, 1),
    #     ResBlock(6, 6, 3, 3, 1, 1),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(6, 10, 3, 1, 1, 1),
    #     ResBlock(10, 10, 3, 3, 1, 1),
    #     lr=0.1,
    #     batch_size=800,
    #     epoch_count=10,
    # ), # 76 - 500
    # create_model(
    #     28,
    #     2,
    #     2,6
    #     nn.Conv2d(1, 3, 3, 1, 1, 1),
    #     ResBlock(3, True, True, 3, 3, 3, 1, 1),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(3, 6, 3, 1, 1, 1),
    #     ResBlock(6, True, True, 6, 3, 3, 1, 1),
    #     lr=0.1,
    #     batch_size=800,
    #     epoch_count=10,
    # ), # 65 - 100
    # create_model(
    #     28,
    #     2,
    #     2,
    #     nn.Conv2d(1, 3, 3, 1, 1, 1),
    #     ResBlock(3, True, True, 3, 3, 3, 1, 1),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(3, 6, 3, 1, 1, 1),
    #     ResBlock(6, True, True, 6, 3, 3, 1, 1),
    #     lr=0.001,
    #     batch_size=30,
    #     epoch_count=10,
    # ), # 81 - 1000
    create_model(
        28,
        2,
        2,
        # ResBlock(1, True, True, 3, 3, 3, 1, 1),
        nn.Conv2d(1, 3, 3, 1, 1, 1),
        ResBlock(3, True, True, 10, 3, 3, 1, 1),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(3, 6, 3, 1, 1),
        ResBlock(6, True, True, 20, 3, 3, 1, 1, 1, 1),
        linears=[],
        lr=0.001,
        batch_size=30,
        # lr=0.001,
        # batch_size=30,
        epoch_count=1,
    ), # 81 - 1000
)

Conv2d_ResBlock_MaxPool2d_Conv2d_ResBlock_MaxPool2d_Flatten_Linear_Softmax


In [58]:
try:
    for model, er_f, optim, writer, name, batch_count, epoch_count in tqdm_notebook(models):
        model = torch.load("88.pth")
        model.eval()
        #  = create_model(28, *param)
        torch.manual_seed(0)
        data_loader = DataLoader(dataset, batch_size=batch_count, shuffle=True)
        total_epochs = 0
        for epoch in tqdm_notebook(range(epoch_count)):
            for image, target in tqdm_notebook(data_loader, leave=False):
                optim.zero_grad()
                outs = model(image)
                loss = er_f(outs, target)
                loss.backward()
                percent: torch.tensor = ((outs.max(1).indices == target.max(1).indices).sum() / len(target))
                optim.step()
                writer.add_scalar("loss", loss, total_epochs)
                writer.add_scalar("train_accuracy", percent, total_epochs)
                predicted = model(target_data)
                predicted_labels = predicted.max(1).indices
                percent: torch.tensor = ((predicted_labels == target_labels).sum() / len(target_labels))
                writer.add_scalar("test_accuracy", percent, total_epochs)
                total_epochs += 1
finally:
    write_summary(writer, model)
    torch.save(model, f"model_{name}.pth")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

/home/alex1um/.local/lib/python3.11/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/alex1um/.local/lib/python3.11/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


KeyboardInterrupt: 

In [59]:
optim = torch.optim.AdamW(model.parameters(), lr=0.01)
data_loader = DataLoader(dataset, batch_size=500, shuffle=True)

In [41]:
optim = torch.optim.AdamW(model.parameters(), lr=0.0001)
data_loader = DataLoader(dataset, batch_size=1000, shuffle=True)

In [60]:
try:
    for epoch in tqdm_notebook(range(10)):
        for image, target in tqdm_notebook(data_loader, leave=False):
            optim.zero_grad()
            outs = model(image)
            loss = er_f(outs, target)
            loss.backward()
            optim.step()
            percent: torch.tensor = ((outs.max(1).indices == target.max(1).indices).sum() / len(target))
            writer.add_scalar("loss", loss, total_epochs)
            writer.add_scalar("train_accuracy", percent, total_epochs)
            predicted = model(target_data)
            predicted_labels = predicted.max(1).indices
            percent: torch.tensor = ((predicted_labels == target_labels).sum() / len(target_labels))
            writer.add_scalar("test_accuracy", percent, total_epochs)
            total_epochs += 1
    write_summary(writer, model)
    torch.save(model, f"model_{name}.pth")
finally:
    write_summary(writer, model)
    torch.save(model, f"model_{name}.pth")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

/home/alex1um/.local/lib/python3.11/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

/home/alex1um/.local/lib/python3.11/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
torch.load(model, f"model_{name}.pth")

In [44]:
torch.save(model, "89.pt")